In [1]:
pip install --upgrade scikit-learn threadpoolctl

  Using cached scikit_learn-1.4.1.post1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached threadpoolctl-3.4.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.4.1.post1-cp311-cp311-win_amd64.whl (10.6 MB)
Using cached threadpoolctl-3.4.0-py3-none-any.whl (17 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# set workspace
WhoIsWorking = "AA"
WorkLocation = "School"

if WhoIsWorking == "RT":
    if WorkLocation == "Home":
        folder_path = r'C:\Users\rentr\Desktop\ENGO\DataMining\ENGO645_TermProject_Data'
    elif WorkLocation == "School":
        folder_path = r'D:\ENGO645_TermProject_Data'
elif WhoIsWorking == "AA":
    folder_path = r'C:\Users\achan\OneDrive\Desktop\Term Project Data'
elif WhoIsWorking == "IA":
    folder_path = r'C:\Users\afoam\assignments\ENGO645_TorontoCrimeAnalysis'
elif WhoIsWorking == "AI":
    folder_path = r'C:\Adewale_Directory\Capacity_Building\MGIS\Lecture_Note\WINTER2024\ENGO645\Course_Project\Project_Data'

print(folder_path)

C:\Users\achan\OneDrive\Desktop\Term Project Data


In [6]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the data
neighbourhood_data = pd.read_csv(r"C:\Users\achan\OneDrive\Desktop\Term Project Data\Neighbourhood_Cleaned.csv")
crime_data = pd.read_csv(r"C:\Users\achan\OneDrive\Desktop\Term Project Data\MCI_Cleaned.csv")

# Merge the two datasets on the neighbourhood column
merged_data = pd.merge(neighbourhood_data, crime_data, left_on='NeighbourhoodName', right_on='NEIGHBOURHOOD_158')

# Select the columns to use for clustering
columns = ['Poles_Count', 'POI_Count', 'TCamera_Count', 'TotalPopulation', 'MedianTotalIncome', 'AverageTotalIncome', 'BuildingCoveragePERCENTAGE', 'ParksOSNAPERCENTAGE', 'BuiltUpPERCENTAGE', 'AveragePrice', 'OCC_YEAR', 'OCC_HOUR', 'TEMP']
data_for_clustering = merged_data[columns]

# Handle missing values
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data_for_clustering)

# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

# Perform K-means clustering
kmeans = KMeans(n_clusters=5, random_state=0)  
clusters = kmeans.fit_predict(data_scaled)

# Add the cluster labels to the original data
merged_data['Cluster'] = clusters

# Identify the high-risk areas
high_risk_areas = merged_data[merged_data['Cluster'] == merged_data['Cluster'].value_counts().idxmin()]

print(high_risk_areas)

                                                 geometry  \
101225  POLYGON ((-79.3526822361769 43.655714324394, -...   
101226  POLYGON ((-79.3526822361769 43.655714324394, -...   
101227  POLYGON ((-79.3526822361769 43.655714324394, -...   
101228  POLYGON ((-79.3526822361769 43.655714324394, -...   
101229  POLYGON ((-79.3526822361769 43.655714324394, -...   
...                                                   ...   
264385  POLYGON ((-79.3452871691637 43.7528896509851, ...   
264386  POLYGON ((-79.3452871691637 43.7528896509851, ...   
264387  POLYGON ((-79.3452871691637 43.7528896509851, ...   
264388  POLYGON ((-79.3452871691637 43.7528896509851, ...   
264389  POLYGON ((-79.3452871691637 43.7528896509851, ...   

                            NeighbourhoodName  Poles_Count  POI_Count  \
101225  St Lawrence-East Bayfront-The Islands         1968       1513   
101226  St Lawrence-East Bayfront-The Islands         1968       1513   
101227  St Lawrence-East Bayfront-The Islands   

In [5]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load the data
neighbourhood_data = pd.read_csv(r"C:\Users\achan\OneDrive\Desktop\Term Project Data\Neighbourhood_Cleaned.csv")
crime_data = pd.read_csv(r"C:\Users\achan\OneDrive\Desktop\Term Project Data\MCI_Cleaned.csv")

# Select relevant features for clustering
features_neighbourhood = neighbourhood_data[['Poles_Count', 'POI_Count', 'TCamera_Count', 'TotalPopulation', 
                                             'MedianTotalIncome', 'AverageTotalIncome', 'BuildingCoveragePERCENTAGE', 
                                             'ParksOSNAPERCENTAGE', 'BuiltUpPERCENTAGE', 'AveragePrice']]

# Drop rows with missing values
features_neighbourhood.dropna(inplace=True)

# Normalize features
scaler = StandardScaler()
scaled_features_neighbourhood = scaler.fit_transform(features_neighbourhood)

# Perform k-means clustering
k = 5  # Number of clusters
kmeans_neighbourhood = KMeans(n_clusters=k, random_state=42)
clusters_neighbourhood = kmeans_neighbourhood.fit_predict(scaled_features_neighbourhood)

# Add cluster labels to the neighbourhood data
neighbourhood_data['Cluster'] = clusters_neighbourhood

# Select relevant features for crime data
features_crime = crime_data[['NEIGHBOURHOOD_158', 'LAT_WGS84', 'LONG_WGS84']]

# Group crime data by neighbourhood and count the number of crimes
crime_counts = features_crime.groupby('NEIGHBOURHOOD_158').size().reset_index(name='Crime_Count')

# Merge crime counts with neighbourhood data
neighbourhood_data = neighbourhood_data.merge(crime_counts, on='NEIGHBOURHOOD_158', how='left')

# Output clusters and crime counts for high-risk areas
high_risk_areas = neighbourhood_data[['NeighbourhoodName', 'Cluster', 'Crime_Count']]
high_risk_areas.to_csv("High_Risk_Areas.csv", index=False)

C:\Users\achan\AppData\Local\Temp\ipykernel_6912\2149622241.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_neighbourhood.dropna(inplace=True)
C:\Users\achan\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KeyError: 'NEIGHBOURHOOD_158'